# Producing merged clean cat

In [2]:
import os, sys
sys.path.append('../')
from xd import *
from utils import *
import esutil
import healpy as hp
from systematics import *
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [3]:
path = '/n/des/lee.5922/data/gold_cat/'


In [4]:

def AddingSFD98ReddeningColumn(cat, kind='SPT', coeff = [3.186,2.140,1.569,1.196 ] ):
    import numpy.lib.recfunctions as rf
    import pandas as pd

    band = ['G', 'R', 'I', 'Z']

    if 'EBV' not in cat.dtype.names :   
     
        print 'Using SFD98 nside 4096 healpix map'
        print 'Bands :',  band
        #print 'NSIDE = 4096'
        print 'coefficients = ', coeff
        nside = 4096

        #from suchyta_utils.y1a1_slr_shiftmap import SLRShift
        #sfdfile = '/n/des/lee.5922/data/systematic_maps/y1a1_wide_slr_wavg_zpshift2.fit'
        mapname = '/n/des/lee.5922/data/systematic_maps/ebv_sfd98_fullres_nside_4096_nest_equatorial.fits'
        #mapname = '/n/des/lee.5922/data/systematic_maps/ebv_lenz17_nside_4096_nest_equatorial.fits'
        reddening_ring = hp.read_map(mapname)
        hpIndices = np.arange(reddening_ring.size)
        #goodmask = hp.mask_good(reddening_ring)
        #goldmask = 

        goodIndices = hpIndices #hpIndices[goodmask]
        clean_map = reddening_ring #reddening_ring[goodmask]

        sysMap = np.zeros((clean_map.size, ), dtype=[('PIXEL', 'i4'), ('EBV', 'f8'), ('RA', 'f8'), ('DEC', 'f8')])
        sysMap['PIXEL'] = goodIndices
        sysMap['EBV'] = clean_map
        
        sys_ra, sys_dec = hpHEALPixelToRaDec(goodIndices, nside = nside)
        sysMap['RA'] = sys_ra
        sysMap['DEC'] = sys_dec

        from cmass_modules.Cuts import keepGoodRegion
        sysMap = keepGoodRegion(sysMap)
        if kind is 'STRIPE82': sysMap = sysMap[sysMap['DEC'] > -30]
        elif kind is 'SPT': sysMap = sysMap[sysMap['DEC'] < -30]


        cat_hp = cat
        hpind = hpRaDecToHEALPixel(cat_hp['RA'], cat_hp['DEC'], nside= 4096, nest= False)
        #cat_hp.dtype.names = [str(x) for x in cat_hp.dtype.names]
        cat_hp = changeColumnName(cat_hp, name = 'HPIX', rename = 'PIXEL')
        cat_hp['PIXEL'] = hpind
        
        #sfdmap = changeColumnName( sysMap_ge, name = 'SIGNAL', rename = 'SFD98' )


        try : 

            cat_Data = pd.DataFrame(cat_hp)
            sfdData = pd.DataFrame(sysMap)
            matched = pd.merge(cat_Data, sfdData, on='PIXEL', how='left', 
                               suffixes = ['','_sys'], left_index=False, right_index=False)
        except ValueError :
            print "Big-endian buffer not supported on little-endian compiler"
            print "Doing byte swapping ...."

            cat_hp = np.array(cat_hp).byteswap().newbyteorder()
            #sfdmap = np.array(sfdmap).byteswap().newbyteorder()
            cat_Data = pd.DataFrame(cat_hp)
            sfdData = pd.DataFrame(sysMap)
            

            #print cat_Data.keys()
            #print sfdData.keys()
            matched = pd.merge(cat_Data, sfdData, on='PIXEL', how='left', 
                               suffixes = ['','_sys'], left_index=False, right_index=False)
            
        matched_arr = matched.to_records(index=False)
        matched_arr.dtype.names = [str(x) for x in matched_arr.dtype.names]

    matched_arr = changeColumnName(matched_arr, name = 'PIXEL', rename = 'HPIX')
    
    return matched_arr

In [4]:
def AddingSLRReddeningColumn(cat):


    if 'SLR_SHIFT_G' not in cat.dtype.names : 
        import numpy.lib.recfunctions as rf   
        #from suchyta_utils.y1a1_slr_shiftmap import SLRShift
        from y1a1_slr_shiftmap import SLRShift
        zpfile = '/n/des/lee.5922/data/systematic_maps/y1a1_wide_slr_wavg_zpshift2.fit'
        slrshift = SLRShift(zpfile, fill_periphery=True)
        offsets_g = slrshift.get_zeropoint_offset('g',cat['RA'],cat['DEC'],interpolate=True)
        offsets_r = slrshift.get_zeropoint_offset('r',cat['RA'],cat['DEC'],interpolate=True)
        offsets_i = slrshift.get_zeropoint_offset('i',cat['RA'],cat['DEC'],interpolate=True)
        offsets_z = slrshift.get_zeropoint_offset('z',cat['RA'],cat['DEC'],interpolate=True)

        offsets = [ offsets_g, offsets_r, offsets_i, offsets_z  ]
        from pandas import DataFrame, concat
        nametag = ['SLR_SHIFT_'+f for f in ['G', 'R', 'I', 'Z'] ]
        catnametag = cat.dtype.names
        try : 
            offsetsdf = DataFrame( offsets, index = nametag ).T
            cat = DataFrame(cat, index = catnametag)
            #del cat['index']
            print 'concatenate two ndarrays'
            cat = concat([cat, offsetsdf], axis=1)
            print 'dataframe to recordarray'
            cat = cat.to_records()
        
        except ValueError :
            print "Big-endian buffer not supported on little-endian compiler"
            print "Doing byte swapping"
            
            #offsetsdf = np.array(offsetsdf).byteswap().newbyteorder()
            cat = np.array(cat).byteswap().newbyteorder()
            offsetsdf = DataFrame( offsets, index = nametag ).T
            cat = DataFrame(cat)

            print 'concatenate two ndarrays'
            cat = concat([cat, offsetsdf], axis=1)
            print 'dataframe to recordarray'
            cat = cat.to_records()
            cat.dtype.names = [str(x) for x in cat.dtype.names]
            
            #matched = pd.merge(desData, goldData, on=key, how=how, suffixes = suffixes, left_index=left_index)
    return cat

In [6]:
io.SearchFitsByName(path = path, keyword = 'Y1A1_GOLD_00')


--------------------------------
 Existing catalog
--------------------------------
Y1A1_GOLD_000001.fits
Y1A1_GOLD_000002.fits
Y1A1_GOLD_000003.fits
Y1A1_GOLD_000004.fits
Y1A1_GOLD_000005.fits
Y1A1_GOLD_000006.fits
Y1A1_GOLD_000007.fits
Y1A1_GOLD_000008.fits
Y1A1_GOLD_000009.fits
Y1A1_GOLD_000010.fits
Y1A1_GOLD_000011.fits
Y1A1_GOLD_000012.fits
Y1A1_GOLD_000013.fits
Y1A1_GOLD_000014.fits
Y1A1_GOLD_000015.fits


['/n/des/lee.5922/data/gold_cat/Y1A1_GOLD_000001.fits',
 '/n/des/lee.5922/data/gold_cat/Y1A1_GOLD_000002.fits',
 '/n/des/lee.5922/data/gold_cat/Y1A1_GOLD_000003.fits',
 '/n/des/lee.5922/data/gold_cat/Y1A1_GOLD_000004.fits',
 '/n/des/lee.5922/data/gold_cat/Y1A1_GOLD_000005.fits',
 '/n/des/lee.5922/data/gold_cat/Y1A1_GOLD_000006.fits',
 '/n/des/lee.5922/data/gold_cat/Y1A1_GOLD_000007.fits',
 '/n/des/lee.5922/data/gold_cat/Y1A1_GOLD_000008.fits',
 '/n/des/lee.5922/data/gold_cat/Y1A1_GOLD_000009.fits',
 '/n/des/lee.5922/data/gold_cat/Y1A1_GOLD_000010.fits',
 '/n/des/lee.5922/data/gold_cat/Y1A1_GOLD_000011.fits',
 '/n/des/lee.5922/data/gold_cat/Y1A1_GOLD_000012.fits',
 '/n/des/lee.5922/data/gold_cat/Y1A1_GOLD_000013.fits',
 '/n/des/lee.5922/data/gold_cat/Y1A1_GOLD_000014.fits',
 '/n/des/lee.5922/data/gold_cat/Y1A1_GOLD_000015.fits']

In [7]:
for i in range(1,16):
    gold = io.SearchAndCallFits(path = path, keyword = 'Y1A1_GOLD_{:06}.fits'.format(i))
    gold = AddingSLRReddeningColumn(gold )
    gold = AddingSFD98ReddeningColumn(gold, kind='SPT')

    esutil.io.write(path + 'Y1A1_GOLD_slr_ebv_{:06}.fits'.format(i), gold)
    print 'fits file save to ', path + 'Y1A1_GOLD_slr_ebv_{:06}.fits'.format(i)


--------------------------------
 calling catalog
--------------------------------
Y1A1_GOLD_000001.fits
Using zpshift file: /n/des/lee.5922/data/systematic_maps/y1a1_wide_slr_wavg_zpshift2.fit
Bands: ['G' 'R' 'I' 'Z' 'Y']
NSIDE = 512 (~6.9 arcmin)
Area = 1904.99 deg^2
Filling in periphery pixels...
Peripheral area:
  G: 91.94 deg^2
  R: 91.94 deg^2
  I: 91.94 deg^2
  Z: 91.43 deg^2
  Y: 97.79 deg^2
Big-endian buffer not supported on little-endian compiler
Doing byte swapping
concatenate two ndarrays
dataframe to recordarray
Using SFD98 nside 4096 healpix map
Bands : ['G', 'R', 'I', 'Z']
coefficients =  [3.186, 2.14, 1.569, 1.196]
NSIDE = 4096
ORDERING = NESTED in fits file
INDXSCHM = IMPLICIT
Ordering converted to RING
fits file save to  /n/des/lee.5922/data/gold_cat/Y1A1_GOLD_slr_ebv_000001.fits

--------------------------------
 calling catalog
--------------------------------
Y1A1_GOLD_000002.fits
Using zpshift file: /n/des/lee.5922/data/systematic_maps/y1a1_wide_slr_wavg_zpshift2

Using zpshift file: /n/des/lee.5922/data/systematic_maps/y1a1_wide_slr_wavg_zpshift2.fit
Bands: ['G' 'R' 'I' 'Z' 'Y']
NSIDE = 512 (~6.9 arcmin)
Area = 1904.99 deg^2
Filling in periphery pixels...
Peripheral area:
  G: 91.94 deg^2
  R: 91.94 deg^2
  I: 91.94 deg^2
  Z: 91.43 deg^2
  Y: 97.79 deg^2
Big-endian buffer not supported on little-endian compiler
Doing byte swapping
concatenate two ndarrays
dataframe to recordarray
Using SFD98 nside 4096 healpix map
Bands : ['G', 'R', 'I', 'Z']
coefficients =  [3.186, 2.14, 1.569, 1.196]
NSIDE = 4096
ORDERING = NESTED in fits file
INDXSCHM = IMPLICIT
Ordering converted to RING
fits file save to  /n/des/lee.5922/data/gold_cat/Y1A1_GOLD_slr_ebv_000011.fits

--------------------------------
 calling catalog
--------------------------------
Y1A1_GOLD_000012.fits
Using zpshift file: /n/des/lee.5922/data/systematic_maps/y1a1_wide_slr_wavg_zpshift2.fit
Bands: ['G' 'R' 'I' 'Z' 'Y']
NSIDE = 512 (~6.9 arcmin)
Area = 1904.99 deg^2
Filling in periphery pixe

In [5]:
gold = io.SearchFitsByName(path = path, keyword = 'Y1A1_GOLD_00')


--------------------------------
 Existing catalog
--------------------------------
Y1A1_GOLD_000001.fits
Y1A1_GOLD_000002.fits
Y1A1_GOLD_000003.fits
Y1A1_GOLD_000004.fits
Y1A1_GOLD_000005.fits
Y1A1_GOLD_000006.fits
Y1A1_GOLD_000007.fits
Y1A1_GOLD_000008.fits
Y1A1_GOLD_000009.fits
Y1A1_GOLD_000010.fits
Y1A1_GOLD_000011.fits
Y1A1_GOLD_000012.fits
Y1A1_GOLD_000013.fits
Y1A1_GOLD_000014.fits
Y1A1_GOLD_000015.fits
